# I will get this model to work today.
today.

In [1]:
import math
from itertools import product
from typing import Optional, Tuple, Callable, Dict, Any, List
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric as tg
import torch_geometric.nn as geom_nn
from torch import Tensor, LongTensor
from torch import scatter
from torch_geometric.datasets import QM9
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.data import Dataset
from torch_geometric.nn.aggr import SumAggregation

In [2]:
def compute_distance(pos1: Tensor, pos2: Tensor) -> float:
    return torch.sum(torch.pow(torch.sub(pos1, pos2), 2)).item()

In [3]:
def complete_edge_distance(data: Data) -> Tensor:
    positions = data['pos']
    complete_edge_distance = torch.zeros(data.num_nodes, data.num_nodes)
    for i, j in product(enumerate(positions), enumerate(positions)):
        complete_edge_distance[i[0]][j[0]] = complete_edge_distance[j[0]][i[0]] = compute_distance(i[1], j[1])
    return complete_edge_distance

In [5]:
def pre_transform(data: Data) -> Data:
    data.complete_edge_distance = complete_edge_distance(data)
    return data

In [9]:
dataset = QM9('QM9/',pre_transform=pre_transform)
dataset.data.y = dataset.data.y[:, self.target_idx].view(-1,1)

loader = DataLoader(dataset, shuffle=True)

Processing...
Using a pre-processed version of the dataset. Please install 'rdkit' to alternatively process the raw data.


65800
65900
66000
66100
66200
66300
66400
66500
66600
66700
66800
66900
67000
67100
67200
67300
67400
67500
67600
67700
67800
67900
68000
68100
68200
68300
68400
68500
68600
68700
68800
68900
69000
69100
69200
69300
69400
69500
69600
69700
69800
69900
70000
70100
70200
70300
70400
70500
70600
70700
70800
70900
71000
71100
71200
71300
71400
71500
71600
71700
71800
71900
72000
72100
72200
72300
72400
72500
72600
72700
72800
72900
73000
73100
73200
73300
73400
73500
73600
73700
73800
73900
74000
74100
74200
74300
74400
74500
74600
74700
74800
74900
75000
75100
75200
75300
75400
75500
75600
75700
75800
75900
76000
76100
76200
76300
76400
76500
76600
76700
76800
76900
77000
77100
77200
77300
77400
77500
77600
77700
77800
77900
78000
78100
78200
78300
78400
78500
78600
78700
78800
78900
79000
79100
79200
79300
79400
79500
79600
79700
79800
79900
80000
80100
80200
80300
80400
80500
80600
80700
80800
80900
81000
81100
81200
81300
81400
81500
81600
81700
81800
81900
82000
82100
82200
82300
8240

KeyboardInterrupt: 

In [20]:
class EVMPLayer(nn.Module):
    def __init__(self, embed_dim: int) -> None:
        def __init__(self) -> None:
            super(EVMPLayer, self).__init__()
            self.embed_dim = embed_dim
            self.act = nn.ReLU()
            message_input_size = 2 * embed_dim + 1
            
            # take in a message tensor of size 2 * embed_dim + 1 and get out a new h_i of size embed_dim
            self.message_mlp = nn.Sequential(nn.Linear(message_input_size, embed_dim), act)
            
            # take in a message tensor of size embed_dim and an original h_i of size embed_dim and get out a new h_i of size embed_dim
            self.node_update_mlp = nn.Sequential(nn.Linear(2 * embed_dim, embed_dim), act)
                
        def make_message(source_index: int, destination_index: int, distance: float) -> Tensor:
            torch.cat((embed_tensor[source_index], embed_tensor[destination_index], distance), dim=-1)
            return self.message_mlp(message_tensor)
        
        def update_node(node_tensor: Tensor, message_tensor: Tensor) -> Tensor:
            return self.update_mlp(node_tensor, message_tensor)
        
        def forward(self, embed_tensor: Tensor, complete_edge_distance: Tensor) -> Tensor:
            new_embed_tensor = torch.zeros_like(embed_tensor)
            # for each molecule in the dataset
            for ix, source in enumerate(embed_tensor):
                # create a tensor that tracks the sum of the messages
                message_sum = torch.zeros_like(source.size())
                # for each other molecule in the dataset
                for jx, target in enumerate(embed_tensor):
                    if ix != jx:
                        # add the message tensor between them to the sum tensor
                        message_sum += make_message(ix, jx, complete_edge_distance[ix,jx])
                # update the tensor that keeps track of all molecule embeddings by making its row ix the new embedding of the molecule
                new_embed_tensor[ix] = update_node(node, message_sum)
            
            return new_embed_tensor
            

In [54]:
class RowWiseFCL(nn.Module): 
    def __init__(self, in_dim: int, out_dim: int) -> None: 
        super(RowWiseFCL, self).__init__() 
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.row_layer = nn.Sequential(nn.Linear(in_dim, out_dim), nn.ReLU())
      
    def forward(self, in_tensor) -> Tensor:
        out_tensor = torch.zeros(in_tensor.size()[0], self.out_dim)
        for ix, row in enumerate(in_tensor):
            out_tensor[ix] = self.row_layer(row)
        return out_tensor

In [ ]:
class EVGNN(nn.Module):
    def __init__(self) -> None:
        super(EVGNN, self).__init__()
        
        loss_function = nn.MSELoss()
        
        embedding = nn.Embedding(118, 32)
        mp1 = EVMPLayer(32)
        mp2 = EVMPLayer(32)
        prediction1 = RowWiseFCL(32, 8) 
        prediction1 = RowWiseFCL(8, 1)

    def forward(self, data = Data) -> float:
        x = embedding(data)
        x = F.normalize(x, p=1, dim=1)
        x = mp1(x, data.complete_edge_distance)
        x = F.normalize(x, p=1, dim=1)
        x = mp2(x, data.complete_edge_distance)
        x = prediction1(x)
        x = prediction2(x)
        U_hat = torch.sum(x)
        
        return U_hat

In [ ]:
def criterion(pred: float, target: float) -> float:
    return torch.mean((pred - target) ** 2)

In [4]:

model = EVGNN()
optim = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 1

# training loop
model.train()
for epoch in range(num_epochs):
    for data in loader:
        U_hat = model(data)
        U = data.y
        loss = criterion(U_hat, U)
        optim.zero_grad()
        loss.backward()
        optim.step()
    
model.eval()

 for data in loader:
        pred = model(data)
        target = data.y
        preds.append(pred)
        targets.append(target)
        test_mae += total_absolute_error(pred, target).item()